In [13]:

import os
import glob
import sys
sys.path.append('../../')
from tifffile import imread, imwrite
import numpy as np
from tqdm import tqdm
from vollseg import Augmentation2D
from vollseg.helpers import crappify_flou_G_P
from pathlib import Path
from scipy.ndimage import gaussian_filter

In [22]:
ImageDir =  '/Users/rando/Downloads/Kymo/Raw/'
LabelDir = '/Users/rando/Downloads/Kymo/BinaryMask/'

ResizeImageDir =  '/Users/rando/Downloads/Kymo/ResizeRaw/'
ResizeLabelDir = '/Users/rando/Downloads/Kymo/ResizeBinaryMask_2/'

Path(ResizeImageDir).mkdir(exist_ok=True)
Path(ResizeLabelDir).mkdir(exist_ok=True)
def Resizer2D(image, SizeX, SizeY):
    
    ResizeImage = np.zeros([SizeY, SizeX])
    j = 0
    for i in range(0, ResizeImage.shape[1]):
        
        if j < image.shape[1]:
           ResizeImage[:image.shape[0],i] = image[:image.shape[0],j]
           j = j + 1
        else:
            j = 0   
        


    j = 0
    for i in range(0, ResizeImage.shape[0]):
        
        if j < image.shape[0]:
           ResizeImage[i,:] = ResizeImage[j,:]
           j = j + 1
        else:
            j = 0  
    

    return ResizeImage

In [23]:
Raw_path = os.path.join(ImageDir, '*tif')
filesRaw = glob.glob(Raw_path)
filesRaw.sort


RealRaw_path = os.path.join(LabelDir, '*tif')
RealfilesRaw = glob.glob(RealRaw_path)
RealfilesRaw.sort
Size = [400,400]
SizeY = Size[0]
SizeX = Size[0]
for fname in filesRaw:


           for Realfname in RealfilesRaw:

                  name = os.path.basename(os.path.splitext(fname)[0])
                  realname = os.path.basename(os.path.splitext(Realfname)[0])

                  if ( name == realname): 

                            #Read image        
                            Image = imread(fname)
                            MaskImage = imread(Realfname)
                            
                            ResizeImage = Resizer2D(Image, SizeX, SizeY)
                            ResizeMask = Resizer2D(MaskImage, SizeX, SizeY)

                            imwrite(ResizeImageDir + '/' + name + '.tif', ResizeImage.astype('float32'))
                            imwrite(ResizeLabelDir + '/' + name + '.tif', ResizeMask.astype('uint16'))


In [24]:
RealRaw_path = os.path.join(ResizeLabelDir, '*tif')
RealfilesRaw = glob.glob(RealRaw_path)
RealfilesRaw.sort
for Realfname in RealfilesRaw:
    
      realname = os.path.basename(os.path.splitext(Realfname)[0])
      MaskImage = gaussian_filter(imread(Realfname),1)
      imwrite(ResizeLabelDir + '/' + realname + '.tif', MaskImage.astype('uint16'))
      